In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight

In [2]:
file_path = "../Data/features_30_sec.csv"  # 训练数据集
data = pd.read_csv(file_path)
data.drop(labels="filename", axis=1, inplace=True)

In [3]:
X = data.iloc[:, :-1].values  # All columns except the last one
y = data["label"].values

In [4]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

scaler = StandardScaler()
X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test_scaled = scaler.transform(X_test)

In [6]:
class_weights = compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train_full), y=y_train_full
)
class_weight_dict = dict(zip(np.unique(y_train_full), class_weights))
sample_weights = np.array(
    [class_weight_dict[class_label] for class_label in y_train_full]
)

In [7]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_accuracies = []
model = xgb.XGBClassifier(
    n_estimators=200, random_state=42, learning_rate=0.01, max_depth=6, subsample=0.8
)

for train_idx, val_idx in kfold.split(X_train_full_scaled, y_train_full):
    X_train, X_val = X_train_full_scaled[train_idx], X_train_full_scaled[val_idx]
    y_train, y_val = y_train_full[train_idx], y_train_full[val_idx]
    train_sample_weights = sample_weights[train_idx]

    model.fit(X_train, y_train, sample_weight=train_sample_weights)
    y_val_pred = model.predict(X_val)
    cv_accuracies.append(accuracy_score(y_val, y_val_pred))

print(f"CV accuracies: {cv_accuracies}")
print(f"Average CV accuracy: {np.mean(cv_accuracies):.4f}")

model.fit(X_train_full_scaled, y_train_full, sample_weight=sample_weights)

y_test_pred = model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\nTest Accuracy: {test_accuracy:.4f}")

cm_test = confusion_matrix(y_test, y_test_pred)
print("\nTest Confusion Matrix:")
print(cm_test)

print("\nTest Classification Report:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

CV accuracies: [0.75625, 0.70625, 0.78125, 0.75, 0.675]
Average CV accuracy: 0.7338

Test Accuracy: 0.7500

Test Confusion Matrix:
[[15  0  2  0  0  1  0  0  2  0]
 [ 0 20  0  0  0  0  0  0  0  0]
 [ 1  0 16  0  0  2  0  0  0  1]
 [ 0  0  2 11  4  0  2  1  0  0]
 [ 1  0  0  1 15  0  1  0  0  2]
 [ 1  2  1  0  0 15  0  0  0  1]
 [ 0  0  0  1  1  0 17  0  1  0]
 [ 0  0  0  1  0  0  0 18  1  0]
 [ 0  0  0  1  2  0  0  4 13  0]
 [ 2  0  2  0  0  3  1  0  2 10]]

Test Classification Report:
              precision    recall  f1-score   support

       blues       0.75      0.75      0.75        20
   classical       0.91      1.00      0.95        20
     country       0.70      0.80      0.74        20
       disco       0.73      0.55      0.63        20
      hiphop       0.68      0.75      0.71        20
        jazz       0.71      0.75      0.73        20
       metal       0.81      0.85      0.83        20
         pop       0.78      0.90      0.84        20
      reggae       0.6